In [ ]:
### Assuming you have cloned the Cycle GAN repo from github: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
### run download_cyclegan_model.sh script to download pretrained summer2winter_yosemite_pretrained model (instructions on github page)
### Create a jupyter notebook and follow the below code snippets

from models import create_model
from options.test_options import TestOptions
import torch
from polygraphy.backend.onnx import fold_constants
import onnx

opt = TestOptions.parse()
opt.netG = 'resnet_9blocks'
opt.norm = 'instance'
opt.checkpoints_dir = './checkpoints' ### directory storing the pre-trained model
opt.name = 'summer2winter_yosemite_pretrained'

model = create_model(opt)
model.setup(opt)

### conversion to onnx
dummy_inp = torch.rand(1, 3, 256, 256)
onnx_filepath = './summer2winter_yosemite.onnx'
torch.onnx.export(model, dummy_inp, onnx_filepath, opset_version=10, input_names = ['inp_img'])

### optional step to do Constant folding which involves pre-computing expressions that do not depend on runtime information
converted_onnx_model = onnx.load('./summer2winter_yosemite.onnx')
constant_folded_model = fold_constants(onnx_model)
onnx.save(constant_folded_model, './summer2winter_yosemite_folded.onnx')

### This will give you converted onnx model, which can itself be used for on-device deployment

In [ ]:
python3 -m venv openvino_env

source openvino_env/bin/activate

python -m pip install --upgrade pip

pip install openvino-dev==2022.1.0. ### tested on this version currently

In [ ]:
mo -h ### This will return the help message for Model Optimizer if installation finished successfully

In [ ]:
### To convert, run the following in your terminal (you can use either onnx model or the folded onnx model)
### input_model - specify your input onnx file
### data_type - use fp16 to convert model to fp16 precision for better performance
### output_dir - directory where your openvino files will be saved

mo --input_model summer2winter_yosemite_folded.onnx --data_type fp16 --output_dir "./summer2winter_yosemite_openvino"

In [ ]:
### Pytorch Model Inference

from util.util import tensor2im
from data.base_dataset import get_transform
from PIL import Image
import numpy as np

input_image_filename = 'Yosemite_summer.jpg'
input_image = Image.open(input_image_filename).convert('RGB')
transformation = get_transform(opt, grayscale = False)
transformed_img = transformation(input_image).unsqueeze(0)

if opt.eval:
  model.eval()
with torch.no_grad():
  output_image = model.netG_A(transformed_img)

output_image = tensor2im(output_image) ### final Pytorch result image

### OpenVino Model Inference
from openvino.inference_engine import IECore, IENetwork
from openvino.runtime import Core

ie_obj = IECore()

openvino_network = ie_obj.read_network(model = 'summer2winter_yosemite_folded.xml', weights = 'summer2winter_yosemite_folded.bin')
executable_network = ie_obj.load_network(network = openvino_network, device='CPU')

openvino_output = executable_network.infer({'inp_img': transformed_img})

arr_key = list(openvino_output.keys())[0] ### get key for putput image
openvino_output_img = torch.from_numpy(np.asarray(openvino_output[arr_key]))
openvino_output_image = tensor2im(openvino_output_img) ### final OpenVino result image

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (30, 30))
plt.subplot(1, 2, 1)
plt.gca().set_title('Pytorch Output', fontsize = 'xx-large')
plt.imshow(output_image)

plt.subplot(1, 2, 2)
plt.gca().set_title('OpenVino Output', fontsize = 'xx-large')
plt.imshow(openvino_output_image)